In the previous `preprocessing_notebook`, we created **separate** tables for each of the currencies. We then **quantized** the time series using aggregations over a `300ms` time window. This **decreased** the size of the data, **unified** the data format for all currencies, and **reduced** the problem to predicting the average price in the next `300ms` window.

In this `building_features_notebook` we will solve the problem of **building** new useful features. To do this, we will explore popular approaches to feature creation, and, given the constraints of our task, build the best approach. **First**, we will understand which features will be useful from the **target** time series. In the **next** `selecting_features_notebook` we will use information on **other currencies** and, finally, **select** the most useful features!

# Approaches

I know 4 basic ways to solve the problem:

**A) smart feature engineering**: using domain knowledge, important features are created by hand, over which a simple (e.g., linear) model is then built 
> easy to further train on-line \
> it's interpretable\
> very fast model inference\
> domain knowledge is needed

**B) semi-automatic feature engineering**: using some heuristics, different kinds of statistics (medians, quantiles, etc.) are computed, over which then treebased models are built

> less demanding of domain knowledge (because of the use of a more complex model, we can afford to build less expressive features)\
> high expressive power\
> fast model inference\
> cannot be quickly retrained on-line\ 
> it's uninterpretable

**C) statistical autoregressive approach**: models like Arima, Prophet, etc.

> fast model inference\
> correct selection of hyperparameters is necessary to build a good model

**D) RNN-like approaches:** recurrent neural networks like LSTM and others

> very heavy models (in terms of training and inference)\
> can show very good results

Due to my limited knowledge of the cryptocurrency market, I am removing the **A)** option. Since we have a lot of data, it will be quite hard to train high quality statistical models (to enumerate hyperparameters). Because of this approach **C)** is also rejected. Our goal is to predict `300ms` ahead, because of the fact that in approach **D)** this is comparable to inference models, it is also removed.

This leaves approach **B)**, in which we need to automatically construct good features. Moreover, because of the limitation on inference and the lack of on-line retraining, our model must work fast enough (there must not be very many features), and also have a prediction horizon comparable to the learning time of the new model *(we must have a good model at every moment, if the model is built longer than its predictions become obsolete, we will not be able to trade).*

# Solution description

**Target:** predict the price of the target currency for the next `300ms`

**Input:** transaction history for a set of currencies (including the target one)

**Limitations:** fast inference and model learning; small number of features

First, we will generate a huge number of statistical features (based on the target table). Then we will use statistical hypotheses to test whether they are important in the forecasting. We discard all unimportant features, and thus obtain `stats_selected_features`. This part will be solved using methods from `tsfresh` library and *block cross validation* of time series. I use block cross-validation to test the importance of features in different time conditions within the same day. You can read more about what block cross validation is in the **DOCS**.

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import extraction_utils
import preprocessing_utils
from tsfresh.feature_extraction import EfficientFCParameters

**Text from the docstring of `extraction_utils.bcv_extract_features`:**

---

Implement the process of block cross validation of time series with
counting of window features within each block:

- divide the entire dataframe evenly into `n_tests` blocks

- inside each block calculate statistics on `n_windows` windows of size `windows_size`

- as a target column make % change in `target_col` to simplify the task for the tree based models

- creates lag features with numbers from `lags`


Depending on the `mode`, window feature evaluations are satisfactorily
either using the `tsfresh` methods (parallel mode) or using the
`window_featurizing` method (default mode). The difference is that the first method can be parallelized, but in
the process of execution, the size of memory can increase many times over
(due to `roll_time_series` function). In the second method, the amount of
memory does not grow, but it will not work to parallelize the process.

---

In [2]:
names = np.array([
    '1000LUNC_USDT_PERP', '1000SHIB_USDT_PERP', '1000XEC_USDT_PERP',
    '1INCH_USDT_PERP', 'AAVE_USDT_PERP', 'ADA_BUSD_PERP', 'ADA_USDT_PERP',
    'ALGO_USDT_PERP', 'ALICE_USDT_PERP', 'ALPHA_USDT_PERP', 'ANC_BUSD_PERP',
    'ANKR_USDT_PERP', 'ANT_USDT_PERP', 'APE_BUSD_PERP', 'APE_USDT_PERP',
    'API3_USDT_PERP', 'APT_USDT_PERP', 'ARPA_USDT_PERP', 'AR_USDT_PERP',
    'ATA_USDT_PERP', 'ATOM_USDT_PERP', 'AUDIO_USDT_PERP', 'AVAX_BUSD_PERP',
    'AVAX_USDT_PERP', 'AXS_USDT_PERP', 'BAKE_USDT_PERP', 'BAL_USDT_PERP',
    'BAND_USDT_PERP', 'BAT_USDT_PERP', 'BCH_USDT_PERP', 'BEL_USDT_PERP',
    'BLZ_USDT_PERP', 'BNB_BUSD_PERP', 'BNB_USDT_PERP', 'BNX_USDT_PERP',
    'BTCDOM_USDT_PERP', 'BTC_BUSD_PERP', 'BTC_USDT_CQ', 'BTC_USDT_PERP',
    'C98_USDT_PERP', 'CELO_USDT_PERP', 'CELR_USDT_PERP', 'CHR_USDT_PERP',
    'CHZ_USDT_PERP', 'COMP_USDT_PERP', 'COTI_USDT_PERP', 'CRV_USDT_PERP',
    'CTK_USDT_PERP', 'CTSI_USDT_PERP', 'CVC_USDT_PERP', 'CVX_USDT_PERP',
    'DAR_USDT_PERP', 'DASH_USDT_PERP', 'DEFI_USDT_PERP', 'DENT_USDT_PERP',
    'DGB_USDT_PERP', 'DODO_BUSD_PERP', 'DOGE_BUSD_PERP', 'DOGE_USDT_PERP',
    'DOT_BUSD_PERP', 'DOT_USDT_PERP', 'DUSK_USDT_PERP', 'DYDX_USDT_PERP',
    'EGLD_USDT_PERP', 'ENJ_USDT_PERP', 'ENS_USDT_PERP', 'EOS_USDT_PERP',
    'ETC_BUSD_PERP', 'ETC_USDT_PERP', 'ETH_BUSD_PERP', 'ETH_USDT_CQ',
    'ETH_USDT_PERP', 'FIL_BUSD_PERP', 'FIL_USDT_PERP', 'FLM_USDT_PERP',
    'FLOW_USDT_PERP', 'FOOTBALL_USDT_PERP', 'FTM_BUSD_PERP', 'FTM_USDT_PERP',
    'GALA_BUSD_PERP', 'GALA_USDT_PERP', 'GAL_BUSD_PERP', 'GAL_USDT_PERP',
    'GMT_BUSD_PERP', 'GMT_USDT_PERP', 'GRT_USDT_PERP', 'GTC_USDT_PERP',
    'HBAR_USDT_PERP', 'HNT_USDT_PERP', 'HOT_USDT_PERP', 'ICP_BUSD_PERP',
    'ICP_USDT_PERP', 'ICX_USDT_PERP', 'IMX_USDT_PERP', 'INJ_USDT_PERP',
    'IOST_USDT_PERP', 'IOTA_USDT_PERP', 'IOTX_USDT_PERP', 'JASMY_USDT_PERP',
    'KAVA_USDT_PERP', 'KLAY_USDT_PERP', 'KNC_USDT_PERP', 'KSM_USDT_PERP',
    'LDO_USDT_PERP', 'LINA_USDT_PERP', 'LINK_BUSD_PERP', 'LINK_USDT_PERP',
    'LIT_USDT_PERP', 'LPT_USDT_PERP', 'LRC_USDT_PERP', 'LTC_USDT_PERP',
    'LUNA2_USDT_PERP', 'MANA_USDT_PERP', 'MASK_USDT_PERP', 'MATIC_BUSD_PERP',
    'MATIC_USDT_PERP', 'MKR_USDT_PERP', 'MTL_USDT_PERP', 'NEAR_BUSD_PERP',
    'NEAR_USDT_PERP', 'NEO_USDT_PERP', 'NKN_USDT_PERP', 'OCEAN_USDT_PERP',
    'OGN_USDT_PERP', 'OMG_USDT_PERP', 'ONE_USDT_PERP', 'ONT_USDT_PERP',
    'OP_USDT_PERP', 'PEOPLE_USDT_PERP', 'QNT_USDT_PERP', 'QTUM_USDT_PERP',
    'RAY_USDT_PERP', 'REEF_USDT_PERP', 'REN_USDT_PERP', 'RLC_USDT_PERP',
    'ROSE_USDT_PERP', 'RSR_USDT_PERP', 'RUNE_USDT_PERP', 'RVN_USDT_PERP',
    'SAND_USDT_PERP', 'SFP_USDT_PERP', 'SKL_USDT_PERP', 'SNX_USDT_PERP',
    'SOL_BUSD_PERP', 'SOL_USDT_PERP', 'SPELL_USDT_PERP', 'SRM_USDT_PERP',
    'STG_USDT_PERP', 'STMX_USDT_PERP', 'STORJ_USDT_PERP', 'SUSHI_USDT_PERP',
    'SXP_USDT_PERP', 'THETA_USDT_PERP', 'TOMO_USDT_PERP', 'TRB_USDT_PERP',
    'TRX_BUSD_PERP', 'TRX_USDT_PERP', 'UNFI_USDT_PERP', 'UNI_BUSD_PERP',
    'UNI_USDT_PERP', 'VET_USDT_PERP', 'WAVES_BUSD_PERP', 'WAVES_USDT_PERP',
    'WOO_USDT_PERP', 'XEM_USDT_PERP', 'XLM_USDT_PERP', 'XMR_USDT_PERP',
    'XRP_BUSD_PERP', 'XRP_USDT_PERP', 'XTZ_USDT_PERP', 'YFI_USDT_PERP',
    'ZEC_USDT_PERP', 'ZEN_USDT_PERP', 'ZIL_USDT_PERP', 'ZRX_USDT_PERP'
],
                 dtype=object)
TARGET_NAME = 'CHZ_USDT_PERP_MIDPRICE'

In [3]:
quantized_df_dict = preprocessing_utils.load_tables(names=list(names)+[TARGET_NAME],
                                              path_from='../data/quantized')

In [4]:
target_table = quantized_df_dict[TARGET_NAME]

In [5]:
target_table.head(3)

,price_mean
event_time,
2022-11-15 00:00:00.000,0.198392
2022-11-15 00:00:00.300,0.198341
2022-11-15 00:00:00.600,NaN


In [6]:
import warnings
warnings.filterwarnings('ignore')

# Block cross validation and feature building

Let's do a block cross-validation. The number of blocks will be 24 (approximately one block for each hour). For each block we will take 1800 (this is approximately the last 9 minutes) windows with a width of a minute.

Since we have not yet reduced the dimensionality of the features to be counted (`tsfresh` generates about 800), the next step takes quite a long time. \
*You can go get a coffee :)*

In [ ]:
%%time
train_list = extraction_utils.bcv_extract_features(
    df=target_table,
    n_blocks=24, 
    target_col='price_mean',
    n_jobs=8,
    n_windows=1800,
    window_size=200,
    lags=list(range(1,11)),
    mode='parallel',
    fc_parameters=EfficientFCParameters()
)

In [9]:
train_list[0].shape

(1800, 793)

In [10]:
test_size = 10
n_jobs = 8

In [11]:
import selection_utils
from tsfresh.feature_selection.relevance import calculate_relevance_table

In [12]:
help(selection_utils.get_fitted_models)

Help on function get_fitted_models in module selection_utils:

get_fitted_models(train_list: List[pandas.core.frame.DataFrame], n_jobs=8) -> List[xgboost.sklearn.XGBRegressor]
    Returns the trained model for each `train_list` dataframe.
    
    :param train_list: training data list
    :param n_jobs: number of cores for parallel learning
    :return: list fitted `XGBRegressor` models



Note that the time interval of our predictions is `300ms`. At the same time we are not able to train our model on-line. This leads us to think that the model we built will be used in practice for **more time**. So we need to estimate the quality of its predictions not only for the next `300ms`, but for several steps ahead. 

For simplicity, we will take 10 intervals of `300ms`, that is, our model will predict the prices behavior for the next 3 seconds. During this time we are already able to train a new model on a part of the received data.

In [13]:
from typing import List
import xgboost
from xgboost import XGBRegressor
import pandas as pd
import time

In [14]:
def get_rmse(
        models: List[xgboost.sklearn.XGBRegressor],
        test_list: List[pd.DataFrame]
) -> List[List[float]]:
    """
    Predictions are made for each model in the `models` and the corresponding
    training dataset. Then rmse are calculated for all lengths of the
    prediction horizon from 1 to `len(test_list)`. That is, we look at the
    rmse of the predictions for the next `300ms`, the next `600ms`, and so on.  
    
    :param models: list of trained XGBRegressors
    :param test_list: test data list
    :return: list containing the list with rmse for all prediction horizons
    """
    assert len(models) == len(test_list), f'len(models)={len(models)}!={len(test_list)}=len(test_list)!'

    length = test_list[0].shape[0]

    losses = []
    for i, model in enumerate(models):
        losses.append([])
        x, y = test_list[i].drop(['target'], axis=1), test_list[i].loc[:, 'target']
        mse = .0
        for l in range(length):
            y_hat = model.predict(x.iloc[l:l + 1])
            mse += (y_hat - y.iloc[l]) ** 2
            losses[-1].append(np.sqrt(mse / (l + 1)))

    return losses

# Size reduction

### using some techniques, we will select the most important features among all the generated `tsfresh`

In [ ]:
%%time
# default features from tsfresh
def_start_time = time.time()
def_train = [df[:-test_size] for df in train_list]
def_test = [df[-test_size:] for df in train_list]
def_models = selection_utils.get_fitted_models(def_train, n_jobs)
def_rmse = get_rmse(def_models, def_test)
def_time = time.time() - def_start_time

Now we will try to reduce the dimensionality of the feature space by calculating the statistical significance of each of them. After that we will select the uncorrelated features with the highest `p_value`. This will help us reduce the dimensionality of the space even more, and will also help us use the `feature_importance` technique in the future. See **DOCS** for details.

In [16]:
help(selection_utils.get_stats)

Help on function get_stats in module selection_utils:

get_stats(blocks: List[pandas.core.frame.DataFrame], n_jobs: int = 1) -> pandas.core.frame.DataFrame
    Using statistical criteria, calculates the significance of the features
     for each block in the list. Then the obtained `p_value`s are averaged.
    
    :param blocks: list of datas with `target` column and the same scheme
    :param n_jobs: the number of cores that can be used in the calculation of stat values
    :return: df with calculated p_value for each of the attributes



In [17]:
help(selection_utils.stats_select_features)

Help on function stats_select_features in module selection_utils:

stats_select_features(relevance_table: pandas.core.frame.DataFrame) -> List[ForwardRef('str')]
    Using a table with the statistical significance of each feature,
    returns only low-correlated relevant features.
    
    It is assumed that the correlated attributes are calls of the same
    function with different parameters. Therefore, all the features are
    factorized by the values of the function arguments, and from each class
    the representative with the lowest `p_value` is selected. Because the
    table is sorted by `p_value`, factorization is easy to implement
    through a set.
    
    :param relevance_table: a table with the calculated features and their
     statistical significance
    :return: a list of names of relevant low-correlated features from
     `relevance_table`



In [ ]:
%%time
# stats selected features
stats_start_time = time.time()
relevance_table = selection_utils.get_stats(train_list, n_jobs=n_jobs)
stats_selected_features = selection_utils.stats_select_features(relevance_table)
stats_train = [df[stats_selected_features + ['target']][:-test_size] for df in train_list]
stats_test = [df[stats_selected_features + ['target']][-test_size:] for df in train_list]
stats_models = selection_utils.get_fitted_models(stats_train, n_jobs)
stats_rmse = get_rmse(stats_models, stats_test)
stats_time = time.time() - stats_start_time

The last step in dimensionality reduction will be `feature_importance` counting and subsequent selection by these indicators. We will use five basic 5 importances from XGBRegressor ('gain', 'weight', 'cover', 'total_gain', 'total_cover'), and we will take the 6th as the `shap` importance. See the **DOCS** for details on how it works.

In [19]:
help(selection_utils.get_importance)

Help on function get_importance in module selection_utils:

get_importance(models: List[xgboost.sklearn.XGBRegressor], train_list: List[pandas.core.frame.DataFrame], mode: str = 'all') -> Dict[str, float]
    Using the built-in feature importance estimation methods within `XGBRegressor`
    and the shap algorithm, it calculates the importance of the features on all
    training data, normalizes and averages them.
    
    :param models: the list of trained models
    :param train_list: the list of training data
    :param mode: importance calculating mode
    :return: dictionary, its keys are the features from the training data,
     and the values are the calculated importance



In [20]:
help(selection_utils.importance_select_features)

Help on function importance_select_features in module selection_utils:

importance_select_features(importance_dict: Dict[str, float], portion: float = 0.8) -> List[Tuple[str, float]]
    According to the values of the importance of the attributes selects
    the best of them, which contain the `portion` % of the importance
    of all the features.
    
    :param importance_dict: a dictionary with the importance of each feature
    :param portion: portion of the importance of all the features to be ensured
    :return: a minimum number of features, the overall importance of which >= portion



In [ ]:
%%time
# importance selected features (we can use stats models to calculate importances)
imps_start_time = time.time()
importances = selection_utils.get_importance(stats_models, stats_train, mode='all') 
imps_selected_features = [el[0] for el in selection_utils.importance_select_features(importances, 0.8)]
imps_train = [df[imps_selected_features + ['target']][:-test_size] for df in train_list]
imps_test = [df[imps_selected_features + ['target']][-test_size:] for df in train_list]
imps_models = selection_utils.get_fitted_models(imps_train, n_jobs)
imps_rmse = get_rmse(imps_models, imps_test)
imps_time = time.time() - imps_start_time

In [22]:
# calculate rmse

# average of all models
avg_def_rmse = np.hstack(def_rmse).mean(axis=1)
avg_stats_rmse = np.hstack(stats_rmse).mean(axis=1)
avg_imps_rmse = np.hstack(imps_rmse).mean(axis=1)


# average of all prediction lengths
global_def_rmse = avg_def_rmse.mean()
global_stats_rmse = avg_stats_rmse.mean()
global_imps_rmse = avg_imps_rmse.mean()

In [23]:
# summarize
rmses = [f'RMSE l={i}' for i in range(1,11)]
results = pd.DataFrame(columns = ['Time (s)', 'RMSE mean'] + rmses)

num_models = 24

results.loc['all features'] = [def_time/num_models]+[global_def_rmse]+list(avg_def_rmse)
results.loc['stats selection'] = [stats_time/num_models]+[global_stats_rmse]+list(avg_stats_rmse)
results.loc['stats+imp selection'] = [imps_time/num_models]+[global_imps_rmse]+list(avg_imps_rmse)


results.values[:] = results.values.round(4)
results['Time (s)'] = results['Time (s)'].round(1)

results.to_csv('../docs/tables/building_features_results.csv')
results

,Time (s),RMSE mean,RMSE l=1,RMSE l=2,RMSE l=3,RMSE l=4,RMSE l=5,RMSE l=6,RMSE l=7,RMSE l=8,RMSE l=9,RMSE l=10
all features,14.4,0.0183,0.0145,0.0156,0.0169,0.0169,0.0175,0.0185,0.0200,0.0197,0.0215,0.0219
stats selection,2.6,0.0195,0.0168,0.0168,0.0186,0.0204,0.0202,0.0202,0.0202,0.0202,0.0208,0.0212
stats+imp selection,2.2,0.0186,0.0158,0.0161,0.0176,0.0186,0.0188,0.0193,0.0190,0.0192,0.0204,0.0212


**Results**

We can see that the features selected through statistical tests allow us to reduce training time by a factor of 6, while losing 4.4% in accuracy on average across all horizons, and 13% over the next `300ms`.

With feature importance we reduce the learning time of the model by a factor of 8.3, losing in accuracy 11.3% and 17.2%.

Now we know which statistical features from `tsfresh` were the most useful for predicting the target. This will help us to make great optimization in the future - when we take into account information about trades in other currencies, we will only calculate **these features**, thus saving a huge amount of resources.

In [24]:
import tsfresh
tsfresh_features = [f for f in imps_selected_features if 'price_mean__' in f]
fc_parameters = tsfresh.feature_extraction.settings.from_columns(tsfresh_features)

In [25]:
fc_parameters

{'price_mean': {'mean_second_derivative_central': None,
  'cwt_coefficients': [{'coeff': 0, 'w': 2, 'widths': (2, 5, 10, 20)}],
  'partial_autocorrelation': [{'lag': 3}],
  'fft_coefficient': [{'attr': 'imag', 'coeff': 70}],
  'variation_coefficient': None,
  'fft_aggregated': [{'aggtype': 'variance'}],
  'ar_coefficient': [{'coeff': 8, 'k': 10}],
  'augmented_dickey_fuller': [{'attr': 'pvalue', 'autolag': 'AIC'}],
  'friedrich_coefficients': [{'coeff': 3, 'm': 3, 'r': 30}],
  'spkt_welch_density': [{'coeff': 2}],
  'binned_entropy': [{'max_bins': 10}],
  'permutation_entropy': [{'dimension': 3, 'tau': 1}],
  'autocorrelation': [{'lag': 3}],
  'max_langevin_fixed_point': [{'m': 3, 'r': 30}],
  'skewness': None,
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0}],
  'kurtosis': None,
  'change_quantiles': [{'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': False, 'qh': 0.2, 'ql': 0.0}],
  'time_reversal_asymmetry_statistic': [{'lag':

In [26]:
len(list(fc_parameters['price_mean'].keys()))

30

In the final `selection_features_notebook` notebook we will use information about prices of other currencies. Information about which features are the most useful will help us significantly reduce training time.